# main_code

In [17]:
import awswrangler  as wr
import pandas as pd
# import pandas.io.json
import pymysql
import datetime
import json


# load sql
sql = '''
select *
from moyo_scraping.mno_scraping_result
where 1=1
'''
# connect to mysql
df = wr.athena.read_sql_query(sql=sql, database="moyo_order")

# parse json , add request_id
request_result = zip(df.request_id, df.result)
parsed_data = []
for (request_id, result) in request_result:
    parsed_data.append({'request_id': request_id, **json.loads(result)})

# normalize json

## scraping_main
df_scraping_main = pd.json_normalize (parsed_data 
                , sep = '_'
                )

bucket = 'moyo-mart'
table_name = 'scraping_main'
res = wr.s3.to_parquet(
    df=df_scraping_main,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)

df_scraping_bill = pd.json_normalize (parsed_data 
                , sep = '_'
                , record_path= ['paymentHistory','billDetails','items'] 
                , meta = ['request_id',['paymentHistory','billedDate']]
                # ,meta_prefix="main_"
                )

bucket = 'moyo-mart'
table_name = 'scraping_bill'
res = wr.s3.to_parquet(
    df=df_scraping_bill,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)

df_scraping_discount = pd.json_normalize (parsed_data 
                , sep = '_'
                , record_path= ['paymentHistory','discountDetails','items'] 
                , meta = ['request_id',['paymentHistory','paidDate']]
                # ,meta_prefix="main_"
                )
#
bucket = 'moyo-mart'
table_name = 'scraping_discount'
res = wr.s3.to_parquet(
    df=df_scraping_discount,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)

df_scraping_usage = pd.json_normalize (parsed_data 
                , sep = '_'
                , record_path= ['mobileUsageHistory'] 
                , meta = ['request_id']
                # ,meta_prefix="main_"
                )

bucket = 'moyo-mart'
table_name = 'scraping_usage'
res = wr.s3.to_parquet(
    df=df_scraping_usage,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)

df_scraping_family = pd.json_normalize (parsed_data 
                , sep = '_'
                , record_path= ['familyBundleMembers'] 
                , meta = ['request_id']
                # ,meta_prefix="main_"
                )

bucket = 'moyo-mart'
table_name = 'scraping_family'
res = wr.s3.to_parquet(
    df=df_scraping_family,
    path=f"s3://{bucket}/{table_name}/",
    dataset=True,
    database="moyo_mart",
    table=table_name,
    mode="overwrite_partitions", 
    # partition_cols=['date_ymd']
)


# https://s3.console.aws.amazon.com/s3/buckets/moyo-mart?region=ap-northeast-2&prefix=ringo_partition_test/&showversions=false


# v1.2 2023.05.23 에 작업함 
# billDetails paymentDetails 관련 변경사항 slack 참고 
## https://themoyo.slack.com/archives/C04VDRJ29B2/p1684821720087819

## main-code_test-sheet

## scraping_main

In [3]:
df_scraping_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1214 entries, 0 to 1213
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   request_id                      1214 non-null   int64  
 1   contract                        0 non-null      float64
 2   paymentHistory                  1214 non-null   object 
 3   mobileUsageHistory              1214 non-null   object 
 4   familyBundleMembers             1214 non-null   object 
 5   profile_name                    1214 non-null   object 
 6   profile_telecom                 1214 non-null   object 
 7   profile_phoneNumber             1214 non-null   object 
 8   mobilePlan_fee                  1214 non-null   int64  
 9   mobilePlan_name                 1214 non-null   object 
 10  mobilePlan_totalDiscountRate    1214 non-null   float64
 11  mobilePlan_totalDiscountAmount  1214 non-null   int64  
 12  contract_name                   88

In [9]:
df_scraping_main.head(2)

,request_id,contract,paymentHistory,mobileUsageHistory,familyBundleMembers,profile_name,profile_telecom,profile_phoneNumber,mobilePlan_fee,mobilePlan_name,mobilePlan_totalDiscountRate,mobilePlan_totalDiscountAmount,contract_name,contract_endDate,contract_startDate,contract_hasExpired,contract_cancellationFee
0,1,NaN,"[{'paidDate': '2023-04-25', 'billedDate': '202...","[{'callUsage': 11.0, 'dataUsage': 187.8, 'year...",[],임*빈,SKT,01071**02**,0,0플랜 슈퍼히어로,0.0,0,NaN,NaN,NaN,NaN,NaN
1,48,NaN,"[{'paidDate': '2023-04-13', 'billedDate': '202...","[{'callUsage': 506.0, 'dataUsage': 6.1, 'yearM...",[],나*구,SKT,01022**31**,0,슬림,0.0,0,선택약정할인제도 (12 개월),2023-07-25,2022-07-25,False,37880.0
